In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from prod_details import product_demand_per_dc, total_operational_costs, shipping_costs_per_dc

# from prod_details import product_demand_per_dc, handling_out_costs_per_dc, shipping_costs_per_dc

In [2]:
# Import warehousingCosts
import import_ipynb
from warehousingCosts import Warehousing
w = Warehousing()
storage_cost = w.storage
handling_in_cost = w.handling_in
handling_out_cost = w.handling_out
total_warehousing_costs = w.warehousing

importing Jupyter notebook from warehousingCosts.ipynb
Storage Costs: 1224181.7518
Total Handling In Costs: 180870
Total Handling Out Costs: 1804220.62
Total Warehousing Costs: 3209272.3718


In [5]:
# Load the data, then define the index of some data to simplify future data indexing.

outbound_data = pd.read_excel("Outbound.xlsx")
outbound_data.index = outbound_data.State
outbound_data = outbound_data.drop(['State', 'Small shipment'], axis=1)

demand_data = pd.read_excel('Demand Forecast.xlsx')
demand_data.index = demand_data.state

unit_data = pd.read_excel('Product Data per State.xlsx')
oc_data = pd.read_excel('Opening-Closing Costs.xlsx')
unit_data.index = unit_data.state
oc_data.index = oc_data.State

costs_data = pd.read_excel('Opening-Closing Costs.xlsx')
costs_data.index = costs_data.State

product_data = pd.read_excel('Product Master Data.xlsx')
product_data.index = product_data.COMM_NAME


In [6]:
# As-Is situation

def total_costs(dc_allocation, as_is_dc=False):
    dc_product_demand_container = product_demand_per_dc(dc_allocation, demand_data, product_data, False)
    dc_outbound_costs = sum(shipping_costs_per_dc(dc_allocation, demand_data, outbound_data))
    op_costs = total_operational_costs(dc_allocation, as_is_dc, costs_data)

    total_costs = total_warehousing_costs + dc_outbound_costs + op_costs
    print('Total Outbound Costs:', dc_outbound_costs)
    print('Total operational Costs:', op_costs)
    print('Total Costs:', total_costs)
    
    
    return total_costs

In [7]:
# As-Is Situation

# Define the as-is DC allocation. The keys are the DC, the values are the states where the corresponding DC's customers are.
as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

# Calculate the total outbound_costs
total_costs(as_is_dc)


Total Outbound Costs: 1872998.2597681799
Total operational Costs: 7000000
Total Costs: 12082270.631568179


12082270.631568179

In [8]:
# Algorithm 1: Greedy Constructive Search
# Allocate dc based on the distance/costs greedily 
greedy_dc_allocation = {}
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

# Calculate the total costs
total_costs(greedy_dc_allocation)
total_operational_costs(greedy_dc_allocation, as_is_dc, costs_data)

# Turns out, even though the shipping costs are lower,
# the total outbound costs are relatively higher since the operating and opening costs are skyrocketed.

Total Outbound Costs: 1673002.2092817298
Total operational Costs: 10000000
Total Costs: 14882274.58108173


11430000.0

In [9]:
# Algorithm 1b: Greedy Constructive Heuristics
# Now, knowing that opening and operating new DC can be costly, we apply the same algorithm without opening a new DC.

greedy_dc_allocation = {}
for state in outbound_data.index:
    current_dc = outbound_data[list(as_is_dc.keys())]
    best_dc = (current_dc.loc[state]).idxmin()
    if best_dc not in as_is_dc:
        print(best_dc)
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

print(greedy_dc_allocation)

total_costs(greedy_dc_allocation)
total_operational_costs(greedy_dc_allocation)

# The shipping costs are lowered again, but the handling costs are increasing slightly.
# It resulted in a less significant decrease.

{'WA': ['AK', 'ID', 'OR', 'WA'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}
Total Outbound Costs: 1694474.33498175
Total operational Costs: 7000000
Total Costs: 11903746.706781749


7000000